In [1]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-05-12 01:02:31--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.181|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [84]:
import requests
import json
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import numpy as np
import re # 텍스트 전처리
import csv, random
from gensim.models import KeyedVectors # 미리 훈련된 단어 벡터 읽기
from moviepy.editor import *
import io
import base64
from IPython.display import HTML
import io

In [63]:
# https://cloud.google.com/storage/docs/reference/libraries?authuser=0

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = cloud_storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = cloud_storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [14]:
FILENAME = "GoogleNews-vectors-negative300.bin.gz"
w2v_mod = KeyedVectors.load_word2vec_format(FILENAME, binary=True, limit=300000)

In [4]:
def translate(input_txt, source="ko", target="en"):
    cloud_url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    r = requests.post(cloud_url,
                      headers={"X-NCP-APIGW-API-KEY-ID":"ikoxgv3qyf",
                               "X-NCP-APIGW-API-KEY":"SfjIg59wfjz43peNO3yLy0XO152qJ7EU4uEGZTAX"},
                      data={"source": source, "target": target, "text": input_txt})
    info = json.loads(r.text)
    return info["message"]["result"]["translatedText"]

def getKeywords(sentence):
    # input = 'hello I ate chicken'
    # output = ['ate', 'chicken']
    speechs = ['JJ','JJR','JJS','RB','RBR','UH','VB','VBD','VBG','VBN','VBP','VBZ','NN','NNS','NNP','NNPS']
    txt = translate(sentence)
    
    sentences = nltk.sent_tokenize(txt)
    data, datas = [], []
    
    for sent in sentences:
        data = data + nltk.pos_tag(nltk.word_tokenize(sent))
    # print(data)
    for word in data:
        if speechs.count(word[1])!= 0:
            # print(word)
            datas.append(word[0])
    # print(datas)
    return datas

In [53]:
# TODO get keywords
# 변수 설명
# idx: word에 해당하는 index에 대해서  motion 번호를 입력해둠
# word: tag 단어들
# path: 각 모션의 path, 추후 api 사용하게 될 시에 필요함

# keywords = ['play', 'cellphone', 'game', 'teammates', 'knowing']
def most_relevant_tag(keywords):
    f = io.open('data.csv', 'r', encoding='utf-8')
    rdr = csv.reader(f)

    words = []
    path = []
    idx = []

    for i, line in enumerate(rdr):
        n = 0
        tmp = line[2:]
        for w in tmp:
            try:
                w2v_mod[w]
            except KeyError:
                pass
            else:
                if w not in words and w != '':
                    words.append(w)
                    idx.append([i])
                    n = n + 1
                elif w != '':
                    idx[words.index(w)].append(i)
        path.append(line[1])

    f.close()


    s_temp = []

    for k, word in enumerate(words):
        s_temp.append([])
        for j, key in enumerate(keywords):
            s_temp[k].append(w2v_mod.similarity(word, key))

    S = np.sum(s_temp, axis=1)/len(keywords) # 모든 keyword에 대해서 유사도를 합한 행렬

    score = [0] * (i+1) # 각 영상들의 tag 점수를 더해서 tag 갯수로 나눈것
    num = [0] * (i+1) # 영상들의 tag 갯수

    for key, p in enumerate(idx):
        for q in p:
            score[q] = score[q] + S[key]
            num[q] = num[q] + 1

    score_norm = [0]*len(score) # 영상들의 tag의 평균 점수

    for r in range(len(score)):
        score_norm[r] = score[r]/num[r]

    m = np.argmax(score_norm) # max index (can know which tag is the max)

    # 해당 index에 해당하는 모션이 많을 경우 랜덤으로 하나 뽑아줌
    # t = random.sample(idx[m],1)

    # 해당 모션에 대해서 path를 받아옴
    # p = path[t[0]]
    p = path[m]
    return str(m)  # return index

In [100]:
def produceVideo(clip_list): 
    # input format = ['ANI_74', 'ANI_67', ....]
    # assume that clips are saved locally as ANI_74.MOV, ANI_67.MOV, ...
    if not clip_list:
        print("NO CLIPS")
        return None
    for i in range(len(clip_list)):
        clip_list[i] = clip_list[i] + ".mp4"
    # print("clip_list = %s" % clip_list)
    temp = clip_list[0]
    for i in range(len(clip_list)):
        if i < len(clip_list)-1:
            temp = concatenateVideo(temp, clip_list[i+1], "temp%s.mp4" % i)
    return temp
                
def concatenateVideo(path_to_clip1='clip1.mp4', path_to_clip2='clip2.mp4', path_to_output='render.mp4'):
    # clip1 + clip2 => output
    clip1 = VideoFileClip(path_to_clip1)
    clip2 = VideoFileClip(path_to_clip2)
    finalrender = concatenate_videoclips([clip1, clip2])
    finalrender.write_videofile(path_to_output, codec='libx264')
    return path_to_output

def showVideo(path_to_video='render.mp4'):
    video = io.open(path_to_video, 'r+b').read()
    encoded = base64.b64encode(video)
    HTML(data='''<video alt="test" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
    </video>'''.format(encoded.decode('ascii')))

In [85]:
# https://console.firebase.google.com/u/0/?hl=ko
# https://cloud.google.com/storage/docs/reference/libraries?authuser=0

import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from google.cloud import storage as cloud_storage

In [86]:
cred = credentials.Certificate('./serviceAccountKey.json')
firebase_admin.initialize_app(credential=cred, options={
    'storageBucket': 'forever-dog-3635a.appspot.com'
}, name='hackathon!')

In [87]:
# https://console.firebase.google.com/u/0/project/forever-dog-3635a/storage/forever-dog-3635a.appspot.com/files?hl=ko
bucket = storage.bucket()
print(bucket)
blob = bucket.blob('diary.txt')
outfile='./diary.txt'
blob.upload_from_filename(outfile)

<Bucket: forever-dog.appspot.com>


Forbidden: 403 POST https://www.googleapis.com/upload/storage/v1/b/forever-dog.appspot.com/o?uploadType=multipart: (u'Request failed with status code', 403, u'Expected one of', 200)

In [111]:
if __name__ == "__main__":
    clips = []
    # Get diary.txt from firebase!
    # download_blob("forever-dog.appspot.com", "diary?", "./diary.txt")
    f = open("diary.txt", 'r')
    for line in f:
        sentence = line.split('\n')[0]
        print("sentence = %s" % sentence)
        keywords = getKeywords(sentence)  # sentence -> keywords
        print("keywords = %s" % keywords)   
        clipIndex = most_relevant_tag(keywords)  # keywords -> video!
        clips.append(clipIndex)
    f.close()
      
    print("clips index = %s" % clips)
    # clips = ['clip1', 'clip2']  # clip 되면 
    # clips = ['67', '40', '11']
    path_to_resultVideo = produceVideo(clips)
    # 964 480
    clip_revised = VideoFileClip(path_to_resultVideo)
    clip_resized = clip_revised.resize((480, 964)) # make the height 360px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
    audio_background = AudioFileClip('background.mp3')
    final_audio = CompositeAudioClip([clip_resized.audio, audio_background])
    final_clip = clip_resized.set_audio(final_audio)
    # clip_resized.write_videofile("movie_resized.mp4")
    final_clip.write_videofile("movie_resized.mp4")
    # video = io.open(path_to_resultVideo, 'r+b').read()
    # encoded = base64.b64encode(video)
    # HTML(data='''<video alt="test" controls>
    # <source src="data:video/mp4;base64,{0}" type="video/mp4" />
    # </video>'''.format(encoded.decode('ascii')))
    
    # upload_blob("forever-dog-3635a.appspot.com", "./render.mp4", "clip")

    # 행동하면서 낭독까지??? videobooth 
    # 비디오부스를 뭐로 분류할지? 소팅 --> 서칭이 안되니까.
    # 거기서도 라벨링은 수동으로 : 감정은 아직. 기본적 행동만. 현재는. 대분류 - 소분류 고민중.
    # ㅇㄹㅇㄹㅇㄹㅇㄹㅇㄹ: 어떻게 한꺼번에 다운???
    # 디자인은 이렇게 하겠습니다.
    
    

sentence = Today was a very happy day.
keywords = [u'Today', u'was', u'very', u'happy', u'day']
sentence = I won the dance festival award.
keywords = [u'won', u'Dance', u'Festival', u'award']
sentence = Everyone clapped their hands.
keywords = [u'Everyone', u'clapped', u'hands']
sentence = It was exciting, Ho !
keywords = [u'was', u'expiring', u'Ho']
sentence = Bye.
keywords = [u'Bye']
clips index = ['67', '40', '11', '23', '38']


chunk:  27%|██▋       | 46/171 [00:00<00:00, 449.64it/s, now=None]

Moviepy - Building video temp0.mp4.
MoviePy - Writing audio in %s


t:   4%|▍         | 7/186 [00:00<00:02, 66.59it/s, now=None]       

MoviePy - Done.
Moviepy - Writing video temp0.mp4



...

chunk:   0%|          | 0/427 [00:00<?, ?it/s, now=None]

Moviepy - Building video movie_resized.mp4.
MoviePy - Writing audio in %s


t:   1%|          | 5/447 [00:00<00:09, 48.87it/s, now=None]       

MoviePy - Done.
Moviepy - Writing video movie_resized.mp4



Moviepy - Done !
Moviepy - video ready movie_resized.mp4
